<a href="https://colab.research.google.com/github/lokkster/NPC-ML-GA/blob/main/XGBR_GA_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost==1.6.1
!pip install scikit-optimize

In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/full data.csv")

In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:5].values
Y = data.iloc[:,5:].values

train_data,test_data,train_targets,test_targets=train_test_split(X,Y,test_size=0.11,random_state=42)

In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

def mae(actual,pred):
     return (mean_absolute_error(actual,pred))
    
from xgboost import XGBRegressor
xgbr=XGBRegressor()

In [ ]:
xgbrne=XGBRegressor(n_estimators=1000,min_child_weight=1,max_depth=7,subsample=0.9,colsample_bytree=.9,learning_rate=0.1)
xgbrne.fit(train_data,train_targets)
predxgbne=xgbrne.predict(test_data)
print(predxgbne)
print(test_targets)
print("mae:",mae(test_targets,predxgbne))

xgbrne.save_model('xgbr.h5')

In [ ]:
from sklearn.metrics import r2_score
r2=r2_score(test_targets,predxgbne,multioutput="variance_weighted")
print(r2)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test_targets,predxgbne))
print("rmse:",rmse)

Loading model

In [ ]:
from xgboost import XGBRegressor
new_model=XGBRegressor()
new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc5in1110areanorm.h5')

Defining inputs of prediction

In [ ]:
def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predc2h4=preddf[2]
   predacetic=preddf[3]
   predeth=preddf[4]
   predvalue=predc2h4+predacetic+predeth
   predarr=predvalue.to_numpy()

   return predarr

Define Fitness

In [ ]:
def fitness(input):
    ans=prediction(input)
    return ans

Generating solution pool

In [ ]:
import random
input=[]
for s in range(30):
    NPCtype=random.randint(1,2)
    NPCweight=random.randrange(28000,75000,50)/1000
    loading=random.randrange(295,495,5)/1000
    KOHconc=random.choice([1,3,0.75])
    current=random.randrange(100,1001,50)


    input.append([(NPCtype,NPCweight,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])

In [ ]:
#select first four as parents(best 4)
def parent_selection(mating_pool):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 5].argsort()[::-1]]
    parents_pool=mixsort[:,0:5]
    parents_results=mixsort[:,5]
    return parents_pool

print(parent_selection(mating_pool))

In [ ]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,E,F,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=E[i]
          E[i]=F[i]
          F[i]=temp
    return A,B,C,D,E,F

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    E=mating_pool[4]
    F=mating_pool[5]
    P=np.random.rand(5)
    crossover_pool=uniform_crossover(A,B,C,D,E,F,P)
    crossover_pool=np.asarray(crossover_pool)
  
    return crossover_pool


In [ ]:
#mutation
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.7,1.3)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

In [ ]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 5].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:30,0:5]
    return new_mating_pool
  



Define GA

In [ ]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p

Run GA

In [ ]:
initial_gen=0
max_gen=1000

import numpy as np
while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)

   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]+test[:,3]

   print(c2test[0])
   if c2test[0]>=70 : break

In [ ]:
initial_gen=0
max_gen=1000

import numpy as np
while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)

   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]+test[:,3]

   print(c2test[0])
   if c2test[0]>=70 : break